# **Predict Restaurant Ratings Using LinearRegression**

In [1]:
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler, OneHotEncoder
from sklearn.compose import ColumnTransformer
from sklearn.pipeline import Pipeline
from sklearn.impute import SimpleImputer
from sklearn.linear_model import LinearRegression
from sklearn.metrics import mean_squared_error, r2_score
import numpy as np

**Load the dataset**

In [2]:
data = pd.read_csv("Dataset .csv")

**Identify features and target variable**

In [3]:
features = data.drop(columns=['Restaurant ID', 'Restaurant Name', 'Aggregate rating', 'Rating color', 'Rating text'])
target = data['Aggregate rating']

**Handle missing values and encode categorical variables**

In [4]:
categorical_features = features.select_dtypes(include=['object']).columns
numeric_features = features.select_dtypes(exclude=['object']).columns

**Create a preprocessor for numeric and categorical features**

In [5]:
numeric_transformer = Pipeline(steps=[
    ('imputer', SimpleImputer(strategy='median')),
    ('scaler', StandardScaler())])

categorical_transformer = Pipeline(steps=[
    ('imputer', SimpleImputer(strategy='most_frequent')),
    ('onehot', OneHotEncoder(handle_unknown='ignore'))])

preprocessor = ColumnTransformer(
    transformers=[
        ('num', numeric_transformer, numeric_features),
        ('cat', categorical_transformer, categorical_features)])

**Split the data into training and testing sets**

In [6]:
X_train, X_test, y_train, y_test = train_test_split(features, target, test_size=0.2, random_state=42)

**Select and train a regression algorithm**

In [7]:
model = Pipeline(steps=[
    ('preprocessor', preprocessor),
    ('regressor', LinearRegression())])
model.fit(X_train, y_train)

Pipeline(steps=[('preprocessor',
                 ColumnTransformer(transformers=[('num',
                                                  Pipeline(steps=[('imputer',
                                                                   SimpleImputer(strategy='median')),
                                                                  ('scaler',
                                                                   StandardScaler())]),
                                                  Index(['Country Code', 'Longitude', 'Latitude', 'Average Cost for two',
       'Price range', 'Votes'],
      dtype='object')),
                                                 ('cat',
                                                  Pipeline(steps=[('imputer',
                                                                   SimpleImputer(strategy='most_frequent')),
                                                                  ('onehot',
                                                                   OneHotEncoder(handle_unknown='ignore'))]),
                                                  Index(['City', 'Address', 'Locality', 'Locality Verbose', 'Cuisines',
       'Currency', 'Has Table booking', 'Has Online delivery',
       'Is delivering now', 'Switch to order menu'],
      dtype='object'))])),
                ('regressor', LinearRegression())])

**Evaluate the model's performance**

In [8]:
y_pred = model.predict(X_test)
mse = mean_squared_error(y_test, y_pred)
r2 = r2_score(y_test, y_pred)

**Interpret the model's results**

In [9]:
# Extract feature importances
coefficients = model.named_steps['regressor'].coef_


**Get feature names from the preprocessor**

In [10]:
categorical_feature_names = model.named_steps['preprocessor'].named_transformers_['cat'].named_steps['onehot'].get_feature_names_out(categorical_features)
feature_names = np.concatenate([numeric_features, categorical_feature_names])


**Sort features by their importance**

In [11]:
feature_importances = pd.DataFrame({'Feature': feature_names, 'Coefficient': coefficients}).sort_values(by='Coefficient', ascending=False)

In [12]:
# Output results
print(f'Mean Squared Error (MSE): {mse}')
print(f'R-squared (R²): {r2}')
print('Top 5 Influential Features:')
print(feature_importances.head())

Mean Squared Error (MSE): 5.54636296412996
R-squared (R²): -1.436773892526042
Top 5 Influential Features:
                                                 Feature  Coefficient
4381   Address_Grand Indonesia Mall, Lantai Ground, E...    25.519296
10188                 Cuisines_Desserts, Bakery, Western    25.519296
4803      Address_Jl. Senopati No. 27, Senopati, Jakarta    22.494256
9392                     Locality Verbose_Tebet, Jakarta     6.651889
8267                                      Locality_Tebet     6.651889
